In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
from scipy.stats import t
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

In [39]:
def read_data(file, y_name):
    df = pd.read_csv(file)
    x = df.loc[:, df.columns != y_name]
    y = df.loc[:, df.columns == y_name]
    return x, y

x, y = read_data("/Users/abjain/Documents/Industry/ML/ML_practice/Project1/ML_heardisease/data/heart.csv",'target')
print(x.shape)
print(x.head())
print(y.shape)
y.head

(303, 13)
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  
0   0     1  
1   0     2  
2   0     2  
3   0     2  
4   0     2  
(303, 1)


<bound method NDFrame.head of      target
0         1
1         1
2         1
3         1
4         1
..      ...
298       0
299       0
300       0
301       0
302       0

[303 rows x 1 columns]>

In [40]:
np.random.seed(177372)

def shuffle(x, y):
    idx = np.random.permutation(x.index)
    x = x.reindex(idx)
    y = y.index(idx)
    return x, y

def data_split(x, y, frac):
    idx = np.random.permutation(x.index)
    train_idx = idx[:int(len(idx)*frac)]
    test_idx = idx[int(len(idx)*frac):len(idx)]

    train_x = x.iloc[train_idx,:]
    train_y = y.iloc[train_idx,:]
    test_x = x.iloc[test_idx,:]
    test_y = y.iloc[test_idx,:]

    return train_x, train_y, test_x, test_y

# 80% data in traing while for testing 20% 
train_x, train_y, test_x, test_y = data_split(x,y,0.8) 

In [41]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(242, 13)
(242, 1)
(61, 13)
(61, 1)


In [42]:
## If you go down in the section of the feature selection we have selected features based on the Lasso regression 
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.2, 
                                                random_state=42)

In [43]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)


(242, 13)
(242, 1)
(61, 13)
(61, 1)
(242, 13)
(242, 1)
(61, 13)
(61, 1)


Scaling the training and testing dataset with the min-max transformation using sklearn. From now we will use sklearn. Since it is much easier and clean

In [44]:
from sklearn import preprocessing
scaler = sklearn.preprocessing.MinMaxScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtest_scaled = scaler.fit_transform(xtest)


Applying the K nearest neighbor using Sklearn

In [ ]:
import pandas as pd
print(pd.DataFrame(xtrain_scaled).head(2))
ytrain

In [56]:
knn_sklearn = KNeighborsClassifier(n_neighbors=3)
knn_sklearn.fit(xtrain_scaled, ytrain)

# Make predictions
predictions_sklearn = knn_sklearn.predict(xtest_scaled)

print("Scikit-learn KNN Predictions:\n", predictions_sklearn)


Scikit-learn KNN Predictions:
 [0 0 1 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0]


/opt/anaconda3/envs/ML/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [37]:
head(train_data_scaled)

NameError: name 'head' is not defined